# Testing Xception

In [2]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
from tuning import callbacks as cb
import datetime
import os
import numpy as np

In [3]:
image_size = (299,299)
batch_size = 64
# Create new datasets
#(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces", image_size, batch_size, preprocess_input)
#(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace", image_size, batch_size, preprocess_input)

# Load datasets from current _split folders
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces_split", image_size, batch_size, None, False, False)
(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace_split", image_size, batch_size, None, False, False)


Found 3916 images belonging to 2 classes.
Found 489 images belonging to 2 classes.
Found 492 images belonging to 2 classes.
Count classes: (2052, 1864, 256, 233, 258, 234)


2022-12-16 09:25:03.015047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43496 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-12-16 09:25:03.018664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43496 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:31:00.0, compute capability: 8.6


Found 78158 images belonging to 2 classes.
Found 9769 images belonging to 2 classes.
Found 9771 images belonging to 2 classes.
Count classes: (36736, 41422, 4592, 5177, 4592, 5179)


## Find learning rate

In [3]:
class_weight = mit.findClassWeights(train_batches)

Weight for class 0: 0.95
Weight for class 1: 1.05


In [3]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()

model.compile(optimizer=tf.keras.optimizers.Adam(), 
                           loss="binary_crossentropy", 
                           metrics="accuracy")


Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

In [4]:
epochs = 200
train_input = []
train_output = []
nr_batches = 10

model.fit(ds_train, steps_per_epoch=nr_batches, callbacks=[cb.LerningRateCallback(100, nr_batches)], epochs=epochs, verbose=2)

Epoch 1/200
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1


2022-12-13 14:44:51.687519: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 14:44:52.782624: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 14:44:57.904202: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


10/10 - 46s - loss: 0.8657 - accuracy: 0.4500 - 46s/epoch - 5s/step
Epoch 2/200
10/10 - 10s - loss: 0.8226 - accuracy: 0.5023 - 10s/epoch - 1s/step
Epoch 3/200
10/10 - 10s - loss: 0.8150 - accuracy: 0.4984 - 10s/epoch - 995ms/step
Epoch 4/200
10/10 - 11s - loss: 0.8162 - accuracy: 0.4886 - 11s/epoch - 1s/step
Epoch 5/200
10/10 - 10s - loss: 0.8409 - accuracy: 0.4766 - 10s/epoch - 971ms/step
Epoch 6/200
10/10 - 10s - loss: 0.8461 - accuracy: 0.4781 - 10s/epoch - 970ms/step
Epoch 7/200
10/10 - 10s - loss: 0.8401 - accuracy: 0.4967 - 10s/epoch - 992ms/step
Epoch 8/200
10/10 - 10s - loss: 0.8071 - accuracy: 0.5031 - 10s/epoch - 995ms/step
Epoch 9/200
10/10 - 9s - loss: 0.8217 - accuracy: 0.4828 - 9s/epoch - 945ms/step
Epoch 10/200
10/10 - 10s - loss: 0.8175 - accuracy: 0.5155 - 10s/epoch - 996ms/step
Epoch 11/200
10/10 - 10s - loss: 0.8348 - accuracy: 0.4734 - 10s/epoch - 1s/step
Epoch 12/200
10/10 - 10s - loss: 0.8563 - accuracy: 0.4523 - 10s/epoch - 1s/step
Epoch 13/200
10/10 - 10s - los

# Museum baseline

## Test Xception Museum

In [5]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight=None, weight=False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_Museum"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_Museum.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 137s 2s/step - loss: 0.8157 - accuracy: 0.5794 - true_positives_21: 1050.0000 - true_negatives_14: 1219.0000 - false_positives_17: 834.0000 - false_negatives_18: 813.0000 - true_positive_rate: 0.5636 - true_negative_rate: 0.5938 - false_positive_rate: 0.4062 - false_negative_rate: 0.4364 - positive_predicted_value: 0.5573 - false_discovery_rate: 0.4427 - negative_predicted_value: 0.5999 - false_omission_rate: 0.4001 - binary_demographic_parity_diff: -148.0000 - demographic_parity: 1884.0000 - binary_equalized_odds_diff: 0.0302 - binary_proportional_parity_diff: -0.0378 - proportional_parity: 0.4811 - binary_predictive_rate_parity_diff: -0.0426 - predictive_rate_parity: 0.5573 - binary_accuracy_parity_diff: 0.1588 - accuracy_parity: 0.5794 - false_negative_rate_par

Epoch 5/20
31/31 [==============================] - 40s 1s/step - loss: 0.3096 - accuracy: 0.8779 - true_positives_21: 1618.0000 - true_negatives_14: 1820.0000 - false_positives_17: 231.0000 - false_negatives_18: 247.0000 - true_positive_rate: 0.8676 - true_negative_rate: 0.8874 - false_positive_rate: 0.1126 - false_negative_rate: 0.1324 - positive_predicted_value: 0.8751 - false_discovery_rate: 0.1249 - negative_predicted_value: 0.8805 - false_omission_rate: 0.1195 - binary_demographic_parity_diff: -218.0000 - demographic_parity: 1849.0000 - binary_equalized_odds_diff: 0.0198 - binary_proportional_parity_diff: -0.0557 - proportional_parity: 0.4722 - binary_predictive_rate_parity_diff: -0.0054 - predictive_rate_parity: 0.8751 - binary_accuracy_parity_diff: 0.7559 - accuracy_parity: 0.8779 - false_negative_rate_parity_diff: -0.0198 - false_positive_rate_parity_diff: 0.0198 - binary_negative_predictive_rate_parity_diff: 0.0054 - negative_predictive_rate_parity: 0.8751 - binary_specificit

In [6]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.524
    
    True positives: 0.000
    False positives: 0.000
    
    True negatives: 258.000
    False negatives: 234.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.000
    True negative rate tn/(tn+fp): 1.000
    
    False negative rate fn/(tp+fn): 1.000
    False positive rate fp/(tn+fp): 0.000
    
    Positive predicted value tp/(tp+fp): 0.000
    False discovery rate fp/(tp+fp): 0.000
    
    Negative predicted value tn/(tn+fn): 0.524
    False omission rate fn/(tn+fn): 0.476
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -492.000
    Demographic parity tp+fp: 0.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -1.000
    
    Binary proportional 

In [7]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), None, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_xcep_baseline"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 131s 2s/step - loss: 0.8937 - accuracy: 0.5790 - true_positives_42: 1050.0000 - true_negatives_28: 1218.0000 - false_positives_34: 825.0000 - false_negatives_36: 824.0000 - true_positive_rate: 0.5603 - true_negative_rate: 0.5962 - false_positive_rate: 0.4038 - false_negative_rate: 0.4397 - positive_predicted_value: 0.5600 - false_discovery_rate: 0.4400 - negative_predicted_value: 0.5965 - false_omission_rate: 0.4035 - binary_demographic_parity_diff: -167.0000 - demographic_parity: 1875.0000 - binary_equalized_odds_diff: 0.0359 - binary_proportional_parity_diff: -0.0426 - proportional_parity: 0.4787 - binary_predictive_rate_parity_diff: -0.0365 - predictive_rate_parity: 0.5600 - binary_accuracy_parity_diff: 0.1580 - accuracy_parity: 0.5790 - false_negative_rate_par

Fold 1...
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 131s 2s/step - loss: 0.9027 - accuracy: 0.5734 - true_positives_42: 1024.0000 - true_negatives_28: 1745.0000 - false_positives_34: 830.0000 - false_negatives_36: 1298.0000 - true_positive_rate: 0.4410 - true_negative_rate: 0.6777 - false_positive_rate: 0.3223 - false_negative_rate: 0.5590 - positive_predicted_value: 0.5523 - false_discovery_rate: 0.4477 - negative_predicted_value: 0.5734 - false_omission_rate: 0.4266 - binary_demographic_parity_diff: -1189.0000 - demographic_parity: 1854.0000 - binary_equalized_odds_diff: 0.2367 - binary_proportional_parity_diff: -0.2428 - proportional_parity: 0.3786 - binary_predictive_rate_parity_diff: -0.0211 - predictive_rate_parity: 0.5523 - binary_accuracy_parity_diff: 0.1309 - accuracy_parity: 0.5654 - false_negat

Fold 2...
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 130s 2s/step - loss: 0.8768 - accuracy: 0.5717 - true_positives_42: 1043.0000 - true_negatives_28: 1711.0000 - false_positives_34: 855.0000 - false_negatives_36: 1289.0000 - true_positive_rate: 0.4473 - true_negative_rate: 0.6668 - false_positive_rate: 0.3332 - false_negative_rate: 0.5527 - positive_predicted_value: 0.5495 - false_discovery_rate: 0.4505 - negative_predicted_value: 0.5703 - false_omission_rate: 0.4297 - binary_demographic_parity_diff: -1102.0000 - demographic_parity: 1898.0000 - binary_equalized_odds_diff: 0.2195 - binary_proportional_parity_diff: -0.2250 - proportional_parity: 0.3875 - binary_predictive_rate_parity_diff: -0.0208 - predictive_rate_parity: 0.5495 - binary_accuracy_parity_diff: 0.1245 - accuracy_parity: 0.5623 - false_negat

Fold 3...
Epoch 1/20
31/31 [==============================] - 132s 2s/step - loss: 0.8099 - accuracy: 0.5952 - true_positives_42: 1082.0000 - true_negatives_28: 1764.0000 - false_positives_34: 807.0000 - false_negatives_36: 1244.0000 - true_positive_rate: 0.4652 - true_negative_rate: 0.6861 - false_positive_rate: 0.3139 - false_negative_rate: 0.5348 - positive_predicted_value: 0.5728 - false_discovery_rate: 0.4272 - negative_predicted_value: 0.5864 - false_omission_rate: 0.4136 - binary_demographic_parity_diff: -1119.0000 - demographic_parity: 1889.0000 - binary_equalized_odds_diff: 0.2209 - binary_proportional_parity_diff: -0.2285 - proportional_parity: 0.3857 - binary_predictive_rate_parity_diff: -0.0136 - predictive_rate_parity: 0.5728 - binary_accuracy_parity_diff: 0.1623 - accuracy_parity: 0.5812 - false_negative_rate_parity_diff: -0.2209 - false_positive_rate_parity_diff: 0.2209 - binary_negative_predictive_rate_parity_diff: 0.0136 - negative_predictive_rate_parity: 0.5728 - bina

Fold 4...
Epoch 1/20
31/31 [==============================] - 143s 2s/step - loss: 0.8918 - accuracy: 0.5750 - true_positives_42: 1049.0000 - true_negatives_28: 1713.0000 - false_positives_34: 856.0000 - false_negatives_36: 1279.0000 - true_positive_rate: 0.4506 - true_negative_rate: 0.6668 - false_positive_rate: 0.3332 - false_negative_rate: 0.5494 - positive_predicted_value: 0.5507 - false_discovery_rate: 0.4493 - negative_predicted_value: 0.5725 - false_omission_rate: 0.4275 - binary_demographic_parity_diff: -1087.0000 - demographic_parity: 1905.0000 - binary_equalized_odds_diff: 0.2162 - binary_proportional_parity_diff: -0.2220 - proportional_parity: 0.3890 - binary_predictive_rate_parity_diff: -0.0219 - predictive_rate_parity: 0.5507 - binary_accuracy_parity_diff: 0.1280 - accuracy_parity: 0.5640 - false_negative_rate_parity_diff: -0.2162 - false_positive_rate_parity_diff: 0.2162 - binary_negative_predictive_rate_parity_diff: 0.0219 - negative_predictive_rate_parity: 0.5507 - bina

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.843+/-0.020
    Loss: 0.379+/-0.033
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.834+/-0.023
    True negative rate tn/(tn+fp): 0.851+/-0.017
    
    False negative rate fn/(tp+fn): 0.166+/-0.023
    False positive rate fp/(tn+fp): 0.149+/-0.017
    
    Positive predicted value tp/(tp+fp): 0.835+/-0.020
    False discovery rate fp/(tp+fp): 0.165+/-0.020
    
    Negative predicted value tn/(tn+fn): 0.850+/-0.020
    False omission rate fn/(tn+fn): 0.150+/-0.020
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.050+/-0.010
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.017+/-0.010
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.017+/-0.010
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.524+/-0.006
    Loss: 0.709+/-0.004
    
    Tes

## Test Xception Museum (weighted) 

In [8]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_Museum_re"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_Museum_re.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
31/31 [==============================] - 143s 2s/step - loss: 0.8827 - accuracy: 0.5779 - true_positives_63: 1103.0000 - true_negatives_42: 1160.0000 - false_positives_51: 871.0000 - false_negatives_54: 782.0000 - true_positive_rate: 0.5851 - true_negative_rate: 0.5711 - false_positive_rate: 0.4289 - false_negative_rate: 0.4149 - positive_predicted_value: 0.5588 - false_discovery_rate: 0.4412 - negative_predicted_value: 0.5973 - false_omission_rate: 0.4027 - binary_demographic_parity_diff: 32.0000 - demographic_parity: 1974.0000 - binary_equalized_odds_diff: -0.0140 - binary_proportional_parity_diff: 0.0082 - proportional_parity: 0.5041 - binary_predictive_rate_parity_diff: -0.0386 - predictive_rate_parity: 0.5588 - binary_accuracy_parity_diff: 0.1558 - accuracy_parity: 0.5779 - 

Make folder...


In [9]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.476
    
    True positives: 234.000
    False positives: 258.000
    
    True negatives: 0.000
    False negatives: 0.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 1.000
    True negative rate tn/(tn+fp): 0.000
    
    False negative rate fn/(tp+fn): 0.000
    False positive rate fp/(tn+fp): 1.000
    
    Positive predicted value tp/(tp+fp): 0.476
    False discovery rate fp/(tp+fp): 0.524
    
    Negative predicted value tn/(tn+fn): 0.000
    False omission rate fn/(tn+fn): 0.000
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 492.000
    Demographic parity tp+fp: 492.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 1.000
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): 1.000
  

In [10]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), class_weight, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_re_xcep"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
31/31 [==============================] - 131s 2s/step - loss: 0.8607 - accuracy: 0.5662 - true_positives_84: 1022.0000 - true_negatives_56: 1196.0000 - false_positives_68: 863.0000 - false_negatives_72: 836.0000 - true_positive_rate: 0.5501 - true_negative_rate: 0.5809 - false_positive_rate: 0.4191 - false_negative_rate: 0.4499 - positive_predicted_value: 0.5422 - false_discovery_rate: 0.4578 - negative_predicted_value: 0.5886 - false_omission_rate: 0.4114 - binary_demographic_parity_diff: -147.0000 - demographic_parity: 1885.0000 - binary_equalized_odds_diff: 0.0308 - binary_proportional_parity_diff: -0.0375 - proportional_parity: 0.4812 - binary_predictive_rate_parity_diff: -0.0464 - predictive_rate_parity: 0.5422 - binary_accuracy_parity_diff: 0.1325 - accuracy_parity: 0.5662 - false_negative_rate_parity_diff: -0.0308 - false_positive_rate_parity_diff: 0.0308 - binary_negative_predictive_rate_parity_diff: 0.0464 - negative_predictive_rate_parity: 0.5422 - binary_specifici

Fold 1...
Epoch 1/20
31/31 [==============================] - 127s 2s/step - loss: 0.8901 - accuracy: 0.5629 - true_positives_84: 1528.0000 - true_negatives_56: 1150.0000 - false_positives_68: 1415.0000 - false_negatives_72: 804.0000 - true_positive_rate: 0.6552 - true_negative_rate: 0.4483 - false_positive_rate: 0.5517 - false_negative_rate: 0.3448 - positive_predicted_value: 0.5192 - false_discovery_rate: 0.4808 - negative_predicted_value: 0.5885 - false_omission_rate: 0.4115 - binary_demographic_parity_diff: 989.0000 - demographic_parity: 2943.0000 - binary_equalized_odds_diff: -0.2069 - binary_proportional_parity_diff: 0.2020 - proportional_parity: 0.6010 - binary_predictive_rate_parity_diff: -0.0693 - predictive_rate_parity: 0.5192 - binary_accuracy_parity_diff: 0.0937 - accuracy_parity: 0.5469 - false_negative_rate_parity_diff: 0.2069 - false_positive_rate_parity_diff: -0.2069 - binary_negative_predictive_rate_parity_diff: 0.0693 - negative_predictive_rate_parity: 0.5192 - binary

Fold 2...
Epoch 1/20
31/31 [==============================] - 130s 2s/step - loss: 0.8637 - accuracy: 0.5592 - true_positives_84: 1511.0000 - true_negatives_56: 1152.0000 - false_positives_68: 1417.0000 - false_negatives_72: 818.0000 - true_positive_rate: 0.6488 - true_negative_rate: 0.4484 - false_positive_rate: 0.5516 - false_negative_rate: 0.3512 - positive_predicted_value: 0.5161 - false_discovery_rate: 0.4839 - negative_predicted_value: 0.5848 - false_omission_rate: 0.4152 - binary_demographic_parity_diff: 958.0000 - demographic_parity: 2928.0000 - binary_equalized_odds_diff: -0.2004 - binary_proportional_parity_diff: 0.1956 - proportional_parity: 0.5978 - binary_predictive_rate_parity_diff: -0.0687 - predictive_rate_parity: 0.5161 - binary_accuracy_parity_diff: 0.0874 - accuracy_parity: 0.5437 - false_negative_rate_parity_diff: 0.2004 - false_positive_rate_parity_diff: -0.2004 - binary_negative_predictive_rate_parity_diff: 0.0687 - negative_predictive_rate_parity: 0.5161 - binary

Fold 3...
Epoch 1/20
31/31 [==============================] - 131s 2s/step - loss: 0.8643 - accuracy: 0.5669 - true_positives_84: 1523.0000 - true_negatives_56: 1172.0000 - false_positives_68: 1377.0000 - false_negatives_72: 825.0000 - true_positive_rate: 0.6486 - true_negative_rate: 0.4598 - false_positive_rate: 0.5402 - false_negative_rate: 0.3514 - positive_predicted_value: 0.5252 - false_discovery_rate: 0.4748 - negative_predicted_value: 0.5869 - false_omission_rate: 0.4131 - binary_demographic_parity_diff: 903.0000 - demographic_parity: 2900.0000 - binary_equalized_odds_diff: -0.1888 - binary_proportional_parity_diff: 0.1844 - proportional_parity: 0.5922 - binary_predictive_rate_parity_diff: -0.0617 - predictive_rate_parity: 0.5252 - binary_accuracy_parity_diff: 0.1007 - accuracy_parity: 0.5503 - false_negative_rate_parity_diff: 0.1888 - false_positive_rate_parity_diff: -0.1888 - binary_negative_predictive_rate_parity_diff: 0.0617 - negative_predictive_rate_parity: 0.5252 - binary

Fold 4...
Epoch 1/20
31/31 [==============================] - 129s 2s/step - loss: 0.8518 - accuracy: 0.5725 - true_positives_84: 1538.0000 - true_negatives_56: 1162.0000 - false_positives_68: 1402.0000 - false_negatives_72: 795.0000 - true_positive_rate: 0.6592 - true_negative_rate: 0.4532 - false_positive_rate: 0.5468 - false_negative_rate: 0.3408 - positive_predicted_value: 0.5231 - false_discovery_rate: 0.4769 - negative_predicted_value: 0.5938 - false_omission_rate: 0.4062 - binary_demographic_parity_diff: 983.0000 - demographic_parity: 2940.0000 - binary_equalized_odds_diff: -0.2060 - binary_proportional_parity_diff: 0.2007 - proportional_parity: 0.6004 - binary_predictive_rate_parity_diff: -0.0706 - predictive_rate_parity: 0.5231 - binary_accuracy_parity_diff: 0.1027 - accuracy_parity: 0.5514 - false_negative_rate_parity_diff: 0.2060 - false_positive_rate_parity_diff: -0.2060 - binary_negative_predictive_rate_parity_diff: 0.0706 - negative_predictive_rate_parity: 0.5231 - binary

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.807+/-0.007
    Loss: 0.431+/-0.015
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.820+/-0.011
    True negative rate tn/(tn+fp): 0.795+/-0.008
    
    False negative rate fn/(tp+fn): 0.180+/-0.011
    False positive rate fp/(tn+fp): 0.205+/-0.008
    
    Positive predicted value tp/(tp+fp): 0.784+/-0.007
    False discovery rate fp/(tp+fp): 0.216+/-0.007
    
    Negative predicted value tn/(tn+fn): 0.829+/-0.008
    False omission rate fn/(tn+fn): 0.171+/-0.008
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.004+/-0.012
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.025+/-0.011
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): 0.025+/-0.011
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.476+/-0.008
    Loss: 0.704+/-0.003
    
    Tes

## Test Xception Museum augmented (unweighted) 

In [11]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(True, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_Museum_aug"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_Mueseum_aug.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
31/31 [==============================] - 148s 2s/step - loss: 0.8992 - accuracy: 0.5531 - true_positives_105: 988.0000 - true_negatives_70: 1178.0000 - false_positives_85: 873.0000 - false_negatives_90: 877.0000 - true_positive_rate: 0.5298 - true_negative_rate: 0.5744 - false_positive_rate: 0.4256 - false_negative_rate: 0.4702 - positive_predicted_value: 0.5309 - false_discovery_rate: 0.4691 - negative_predicted_value: 0.5732 - false_omission_rate: 0.4268 - binary_demographic_parity_diff: -194.0000 - demographic_parity: 1861.0000 - binary_equalized_odds_diff: 0.0446 - binary_proportional_parity_diff: -0.0495 - proportional_parity: 0.4752 - binary_predictive_rate_parity_diff: -0.0423 - predictive_rate_parity: 0.5309 - binary_accuracy_parity_diff: 0.1062 - accuracy_parity: 0.5531 

Epoch 5/20
31/31 [==============================] - 40s 1s/step - loss: 0.6690 - accuracy: 0.6417 - true_positives_105: 1140.0000 - true_negatives_70: 1373.0000 - false_positives_85: 675.0000 - false_negatives_90: 728.0000 - true_positive_rate: 0.6103 - true_negative_rate: 0.6704 - false_positive_rate: 0.3296 - false_negative_rate: 0.3897 - positive_predicted_value: 0.6281 - false_discovery_rate: 0.3719 - negative_predicted_value: 0.6535 - false_omission_rate: 0.3465 - binary_demographic_parity_diff: -286.0000 - demographic_parity: 1815.0000 - binary_equalized_odds_diff: 0.0601 - binary_proportional_parity_diff: -0.0730 - proportional_parity: 0.4635 - binary_predictive_rate_parity_diff: -0.0254 - predictive_rate_parity: 0.6281 - binary_accuracy_parity_diff: 0.2835 - accuracy_parity: 0.6417 - false_negative_rate_parity_diff: -0.0601 - false_positive_rate_parity_diff: 0.0601 - binary_negative_predictive_rate_parity_diff: 0.0254 - negative_predictive_rate_parity: 0.6281 - binary_specifici

In [12]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.524
    
    True positives: 0.000
    False positives: 0.000
    
    True negatives: 258.000
    False negatives: 234.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.000
    True negative rate tn/(tn+fp): 1.000
    
    False negative rate fn/(tp+fn): 1.000
    False positive rate fp/(tn+fp): 0.000
    
    Positive predicted value tp/(tp+fp): 0.000
    False discovery rate fp/(tp+fp): 0.000
    
    Negative predicted value tn/(tn+fn): 0.524
    False omission rate fn/(tn+fn): 0.476
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -492.000
    Demographic parity tp+fp: 0.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -1.000
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -1.000
 

In [13]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    model = m.build_model(True, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), None, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_aug_xcep"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
31/31 [==============================] - 133s 2s/step - loss: 0.9092 - accuracy: 0.5517 - true_positives_126: 984.0000 - true_negatives_84: 1177.0000 - false_positives_102: 884.0000 - false_negatives_108: 872.0000 - true_positive_rate: 0.5302 - true_negative_rate: 0.5711 - false_positive_rate: 0.4289 - false_negative_rate: 0.4698 - positive_predicted_value: 0.5268 - false_discovery_rate: 0.4732 - negative_predicted_value: 0.5744 - false_omission_rate: 0.4256 - binary_demographic_parity_diff: -181.0000 - demographic_parity: 1868.0000 - binary_equalized_odds_diff: 0.0409 - binary_proportional_parity_diff: -0.0462 - proportional_parity: 0.4769 - binary_predictive_rate_parity_diff: -0.0477 - predictive_rate_parity: 0.5268 - binary_accuracy_parity_diff: 0.1034 - accuracy_parity: 0.5517 - false_negative_rate_parity_diff: -0.0409 - false_positive_rate_parity_diff: 0.0409 - binary_negative_predictive_rate_parity_diff: 0.0477 - negative_predictive_rate_parity: 0.5268 - binary_specifi

Fold 1...
Epoch 1/20
31/31 [==============================] - 131s 2s/step - loss: 0.9088 - accuracy: 0.5583 - true_positives_126: 1003.0000 - true_negatives_84: 1689.0000 - false_positives_102: 878.0000 - false_negatives_108: 1327.0000 - true_positive_rate: 0.4305 - true_negative_rate: 0.6580 - false_positive_rate: 0.3420 - false_negative_rate: 0.5695 - positive_predicted_value: 0.5332 - false_discovery_rate: 0.4668 - negative_predicted_value: 0.5600 - false_omission_rate: 0.4400 - binary_demographic_parity_diff: -1135.0000 - demographic_parity: 1881.0000 - binary_equalized_odds_diff: 0.2275 - binary_proportional_parity_diff: -0.2318 - proportional_parity: 0.3841 - binary_predictive_rate_parity_diff: -0.0268 - predictive_rate_parity: 0.5332 - binary_accuracy_parity_diff: 0.0994 - accuracy_parity: 0.5497 - false_negative_rate_parity_diff: -0.2275 - false_positive_rate_parity_diff: 0.2275 - binary_negative_predictive_rate_parity_diff: 0.0268 - negative_predictive_rate_parity: 0.5332 - b

Fold 2...
Epoch 1/20
31/31 [==============================] - 131s 2s/step - loss: 0.8882 - accuracy: 0.5641 - true_positives_126: 1049.0000 - true_negatives_84: 1665.0000 - false_positives_102: 887.0000 - false_negatives_108: 1297.0000 - true_positive_rate: 0.4471 - true_negative_rate: 0.6524 - false_positive_rate: 0.3476 - false_negative_rate: 0.5529 - positive_predicted_value: 0.5418 - false_discovery_rate: 0.4582 - negative_predicted_value: 0.5621 - false_omission_rate: 0.4379 - binary_demographic_parity_diff: -1026.0000 - demographic_parity: 1936.0000 - binary_equalized_odds_diff: 0.2053 - binary_proportional_parity_diff: -0.2095 - proportional_parity: 0.3953 - binary_predictive_rate_parity_diff: -0.0203 - predictive_rate_parity: 0.5418 - binary_accuracy_parity_diff: 0.1082 - accuracy_parity: 0.5541 - false_negative_rate_parity_diff: -0.2053 - false_positive_rate_parity_diff: 0.2053 - binary_negative_predictive_rate_parity_diff: 0.0203 - negative_predictive_rate_parity: 0.5418 - b

...

Epoch 9/20
31/31 [==============================] - 39s 1s/step - loss: 0.5914 - accuracy: 0.6955 - true_positives_126: 1239.0000 - true_negatives_84: 1486.0000 - false_positives_102: 562.0000 - false_negatives_108: 631.0000 - true_positive_rate: 0.6626 - true_negative_rate: 0.7256 - false_positive_rate: 0.2744 - false_negative_rate: 0.3374 - positive_predicted_value: 0.6880 - false_discovery_rate: 0.3120 - negative_predicted_value: 0.7019 - false_omission_rate: 0.2981 - binary_demographic_parity_diff: -316.0000 - demographic_parity: 1801.0000 - binary_equalized_odds_diff: 0.0630 - binary_proportional_parity_diff: -0.0807 - proportional_parity: 0.4597 - binary_predictive_rate_parity_diff: -0.0140 - predictive_rate_parity: 0.6880 - binary_accuracy_parity_diff: 0.3910 - accuracy_parity: 0.6955 - false_negative_rate_parity_diff: -0.0630 - false_positive_rate_parity_diff: 0.0630 - binary_negative_predictive_rate_parity_diff: 0.0140 - negative_predictive_rate_parity: 0.6880 - binary_specifi

Fold 3...
Epoch 1/20
31/31 [==============================] - 135s 2s/step - loss: 0.8836 - accuracy: 0.5559 - true_positives_126: 1382.0000 - true_negatives_84: 1290.0000 - false_positives_102: 1263.0000 - false_negatives_108: 962.0000 - true_positive_rate: 0.5896 - true_negative_rate: 0.5053 - false_positive_rate: 0.4947 - false_negative_rate: 0.4104 - positive_predicted_value: 0.5225 - false_discovery_rate: 0.4775 - negative_predicted_value: 0.5728 - false_omission_rate: 0.4272 - binary_demographic_parity_diff: 393.0000 - demographic_parity: 2645.0000 - binary_equalized_odds_diff: -0.0843 - binary_proportional_parity_diff: 0.0803 - proportional_parity: 0.5401 - binary_predictive_rate_parity_diff: -0.0503 - predictive_rate_parity: 0.5225 - binary_accuracy_parity_diff: 0.0913 - accuracy_parity: 0.5456 - false_negative_rate_parity_diff: 0.0843 - false_positive_rate_parity_diff: -0.0843 - binary_negative_predictive_rate_parity_diff: 0.0503 - negative_predictive_rate_parity: 0.5225 - bin

Epoch 5/20
31/31 [==============================] - 38s 1s/step - loss: 0.6475 - accuracy: 0.6588 - true_positives_126: 1188.0000 - true_negatives_84: 1393.0000 - false_positives_102: 642.0000 - false_negatives_108: 695.0000 - true_positive_rate: 0.6309 - true_negative_rate: 0.6845 - false_positive_rate: 0.3155 - false_negative_rate: 0.3691 - positive_predicted_value: 0.6492 - false_discovery_rate: 0.3508 - negative_predicted_value: 0.6671 - false_omission_rate: 0.3329 - binary_demographic_parity_diff: -258.0000 - demographic_parity: 1830.0000 - binary_equalized_odds_diff: 0.0536 - binary_proportional_parity_diff: -0.0658 - proportional_parity: 0.4671 - binary_predictive_rate_parity_diff: -0.0180 - predictive_rate_parity: 0.6492 - binary_accuracy_parity_diff: 0.3175 - accuracy_parity: 0.6588 - false_negative_rate_parity_diff: -0.0536 - false_positive_rate_parity_diff: 0.0536 - binary_negative_predictive_rate_parity_diff: 0.0180 - negative_predictive_rate_parity: 0.6492 - binary_specifi

Fold 4...
Epoch 1/20
31/31 [==============================] - 127s 2s/step - loss: 0.8541 - accuracy: 0.5602 - true_positives_126: 1045.0000 - true_negatives_84: 1681.0000 - false_positives_102: 908.0000 - false_negatives_108: 1263.0000 - true_positive_rate: 0.4528 - true_negative_rate: 0.6493 - false_positive_rate: 0.3507 - false_negative_rate: 0.5472 - positive_predicted_value: 0.5351 - false_discovery_rate: 0.4649 - negative_predicted_value: 0.5710 - false_omission_rate: 0.4290 - binary_demographic_parity_diff: -991.0000 - demographic_parity: 1953.0000 - binary_equalized_odds_diff: 0.1965 - binary_proportional_parity_diff: -0.2024 - proportional_parity: 0.3988 - binary_predictive_rate_parity_diff: -0.0359 - predictive_rate_parity: 0.5351 - binary_accuracy_parity_diff: 0.1133 - accuracy_parity: 0.5567 - false_negative_rate_parity_diff: -0.1965 - false_positive_rate_parity_diff: 0.1965 - binary_negative_predictive_rate_parity_diff: 0.0359 - negative_predictive_rate_parity: 0.5351 - bi

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.662+/-0.030
    Loss: 0.640+/-0.039
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.629+/-0.035
    True negative rate tn/(tn+fp): 0.691+/-0.028
    
    False negative rate fn/(tp+fn): 0.371+/-0.035
    False positive rate fp/(tn+fp): 0.309+/-0.028
    
    Positive predicted value tp/(tp+fp): 0.649+/-0.034
    False discovery rate fp/(tp+fp): 0.351+/-0.034
    
    Negative predicted value tn/(tn+fn): 0.673+/-0.027
    False omission rate fn/(tn+fn): 0.327+/-0.027
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.077+/-0.014
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.062+/-0.015
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.062+/-0.015
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.519+/-0.013
    Loss: 0.693+/-0.002
    
    Tes

# FairFace

## Test Xception FairFace (unweighted) 

In [14]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_FairFace"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_FairFace.h5")

Weight for class 0: 1.06
Weight for class 1: 0.94
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
611/611 [==============================] - 883s 1s/step - loss: 0.6042 - accuracy: 0.6912 - true_positives_147: 30016.0000 - true_negatives_98: 24010.0000 - false_positives_119: 12726.0000 - false_negatives_126: 11406.0000 - true_positive_rate: 0.7246 - true_negative_rate: 0.6536 - false_positive_rate: 0.3464 - false_negative_rate: 0.2754 - positive_predicted_value: 0.7023 - false_discovery_rate: 0.2977 - negative_predicted_value: 0.6779 - false_omission_rate: 0.3221 - binary_demographic_parity_diff: 7326.0000 - demographic_parity: 42742.0000 - binary_equalized_odds_diff: -0.0711 - binary_proportional_parity_diff: 0.0937 - proportional_parity: 0.5469 - binary_predictive_rate_parity_diff: 0.0243 - predictive_rate_parity: 0.7023 - binary_accuracy_parity_diff: 0.3825 - accuracy_pari

Epoch 5/20
611/611 [==============================] - 658s 1s/step - loss: 0.1406 - accuracy: 0.9441 - true_positives_147: 39215.0000 - true_negatives_98: 34574.0000 - false_positives_119: 2162.0000 - false_negatives_126: 2207.0000 - true_positive_rate: 0.9467 - true_negative_rate: 0.9411 - false_positive_rate: 0.0589 - false_negative_rate: 0.0533 - positive_predicted_value: 0.9477 - false_discovery_rate: 0.0523 - negative_predicted_value: 0.9400 - false_omission_rate: 0.0600 - binary_demographic_parity_diff: 4596.0000 - demographic_parity: 41377.0000 - binary_equalized_odds_diff: -0.0056 - binary_proportional_parity_diff: 0.0588 - proportional_parity: 0.5294 - binary_predictive_rate_parity_diff: 0.0078 - predictive_rate_parity: 0.9477 - binary_accuracy_parity_diff: 0.8882 - accuracy_parity: 0.9441 - false_negative_rate_parity_diff: 0.0056 - false_positive_rate_parity_diff: -0.0056 - binary_negative_predictive_rate_parity_diff: -0.0078 - negative_predictive_rate_parity: 0.9477 - binary

In [15]:
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.844
    
    True positives: 4276.000
    False positives: 621.000
    
    True negatives: 3971.000
    False negatives: 903.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.826
    True negative rate tn/(tn+fp): 0.865
    
    False negative rate fn/(tp+fn): 0.174
    False positive rate fp/(tn+fp): 0.135
    
    Positive predicted value tp/(tp+fp): 0.873
    False discovery rate fp/(tp+fp): 0.127
    
    Negative predicted value tn/(tn+fn): 0.815
    False omission rate fn/(tn+fn): 0.185
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 23.000
    Demographic parity tp+fp: 4897.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.039
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): 0

## Test Xception FairFace (weighted)

In [5]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_FairFace_re"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_FairFace_re.h5")

Weight for class 0: 1.06
Weight for class 1: 0.94
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
611/611 [==============================] - 798s 1s/step - loss: 0.6107 - accuracy: 0.6833 - true_positives_21: 28406.0000 - true_negatives_14: 24999.0000 - false_positives_17: 11737.0000 - false_negatives_18: 13016.0000 - true_positive_rate: 0.6858 - true_negative_rate: 0.6805 - false_positive_rate: 0.3195 - false_negative_rate: 0.3142 - positive_predicted_value: 0.7076 - false_discovery_rate: 0.2924 - negative_predicted_value: 0.6576 - false_omission_rate: 0.3424 - binary_demographic_parity_diff: 2128.0000 - demographic_parity: 40143.0000 - binary_equalized_odds_diff: -0.0053 - binary_proportional_parity_diff: 0.0272 - proportional_parity: 0.5136 - binary_predictive_rate_parity_diff: 0.0500 - predictive_rate_parity: 0.7076 - binary_accuracy_parity_diff: 0.3666 - accuracy_parity: 0.6833 - false_negative_

Epoch 5/20
611/611 [==============================] - 698s 1s/step - loss: 0.1457 - accuracy: 0.9417 - true_positives_21: 38948.0000 - true_negatives_14: 34656.0000 - false_positives_17: 2080.0000 - false_negatives_18: 2474.0000 - true_positive_rate: 0.9403 - true_negative_rate: 0.9434 - false_positive_rate: 0.0566 - false_negative_rate: 0.0597 - positive_predicted_value: 0.9493 - false_discovery_rate: 0.0507 - negative_predicted_value: 0.9334 - false_omission_rate: 0.0666 - binary_demographic_parity_diff: 3898.0000 - demographic_parity: 41028.0000 - binary_equalized_odds_diff: 0.0031 - binary_proportional_parity_diff: 0.0499 - proportional_parity: 0.5249 - binary_predictive_rate_parity_diff: 0.0159 - predictive_rate_parity: 0.9493 - binary_accuracy_parity_diff: 0.8835 - accuracy_parity: 0.9417 - false_negative_rate_parity_diff: -0.0031 - false_positive_rate_parity_diff: 0.0031 - binary_negative_predictive_rate_parity_diff: -0.0159 - negative_predictive_rate_parity: 0.9493 - binary_spe

In [6]:
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.850
    
    True positives: 4495.000
    False positives: 780.000
    
    True negatives: 3812.000
    False negatives: 684.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.868
    True negative rate tn/(tn+fp): 0.830
    
    False negative rate fn/(tp+fn): 0.132
    False positive rate fp/(tn+fp): 0.170
    
    Positive predicted value tp/(tp+fp): 0.852
    False discovery rate fp/(tp+fp): 0.148
    
    Negative predicted value tn/(tn+fn): 0.848
    False omission rate fn/(tn+fn): 0.152
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 779.000
    Demographic parity tp+fp: 5275.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.038
    
    Binary propor

## Test Xception FairFace Augmented (unweighted)

In [9]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(True, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_FairFace_aug"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_FairFace_aug.h5")

Weight for class 0: 1.06
Weight for class 1: 0.94
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
611/611 [==============================] - 765s 1s/step - loss: 0.6818 - accuracy: 0.6322 - true_positives_122: 27994.0000 - true_negatives_80: 21419.0000 - false_positives_98: 15317.0000 - false_negatives_104: 13428.0000 - true_positive_rate: 0.6758 - true_negative_rate: 0.5831 - false_positive_rate: 0.4169 - false_negative_rate: 0.3242 - positive_predicted_value: 0.6463 - false_discovery_rate: 0.3537 - negative_predicted_value: 0.6147 - false_omission_rate: 0.3853 - binary_demographic_parity_diff: 8464.0000 - demographic_parity: 43311.0000 - binary_equalized_odds_diff: -0.0928 - binary_proportional_parity_diff: 0.1083 - proportional_parity: 0.5541 - binary_predictive_rate_parity_diff: 0.0317 - predictive_rate_parity: 0.6463 - binary_accuracy_parity_diff: 0.2644 - accuracy_parit

Epoch 5/20
611/611 [==============================] - 646s 1s/step - loss: 0.3728 - accuracy: 0.8301 - true_positives_122: 34902.0000 - true_negatives_80: 29980.0000 - false_positives_98: 6756.0000 - false_negatives_104: 6520.0000 - true_positive_rate: 0.8426 - true_negative_rate: 0.8161 - false_positive_rate: 0.1839 - false_negative_rate: 0.1574 - positive_predicted_value: 0.8378 - false_discovery_rate: 0.1622 - negative_predicted_value: 0.8214 - false_omission_rate: 0.1786 - binary_demographic_parity_diff: 5158.0000 - demographic_parity: 41658.0000 - binary_equalized_odds_diff: -0.0265 - binary_proportional_parity_diff: 0.0660 - proportional_parity: 0.5330 - binary_predictive_rate_parity_diff: 0.0165 - predictive_rate_parity: 0.8378 - binary_accuracy_parity_diff: 0.6603 - accuracy_parity: 0.8301 - false_negative_rate_parity_diff: 0.0265 - false_positive_rate_parity_diff: -0.0265 - binary_negative_predictive_rate_parity_diff: -0.0165 - negative_predictive_rate_parity: 0.8378 - binary_

Epoch 9/20
611/611 [==============================] - 650s 1s/step - loss: 0.2845 - accuracy: 0.8712 - true_positives_122: 36315.0000 - true_negatives_80: 31774.0000 - false_positives_98: 4962.0000 - false_negatives_104: 5107.0000 - true_positive_rate: 0.8767 - true_negative_rate: 0.8649 - false_positive_rate: 0.1351 - false_negative_rate: 0.1233 - positive_predicted_value: 0.8798 - false_discovery_rate: 0.1202 - negative_predicted_value: 0.8615 - false_omission_rate: 0.1385 - binary_demographic_parity_diff: 4396.0000 - demographic_parity: 41277.0000 - binary_equalized_odds_diff: -0.0118 - binary_proportional_parity_diff: 0.0562 - proportional_parity: 0.5281 - binary_predictive_rate_parity_diff: 0.0183 - predictive_rate_parity: 0.8798 - binary_accuracy_parity_diff: 0.7423 - accuracy_parity: 0.8712 - false_negative_rate_parity_diff: 0.0118 - false_positive_rate_parity_diff: -0.0118 - binary_negative_predictive_rate_parity_diff: -0.0183 - negative_predictive_rate_parity: 0.8798 - binary_

Epoch 13/20
611/611 [==============================] - 644s 1s/step - loss: 0.2443 - accuracy: 0.8918 - true_positives_122: 37124.0000 - true_negatives_80: 32577.0000 - false_positives_98: 4159.0000 - false_negatives_104: 4298.0000 - true_positive_rate: 0.8962 - true_negative_rate: 0.8868 - false_positive_rate: 0.1132 - false_negative_rate: 0.1038 - positive_predicted_value: 0.8993 - false_discovery_rate: 0.1007 - negative_predicted_value: 0.8834 - false_omission_rate: 0.1166 - binary_demographic_parity_diff: 4408.0000 - demographic_parity: 41283.0000 - binary_equalized_odds_diff: -0.0095 - binary_proportional_parity_diff: 0.0564 - proportional_parity: 0.5282 - binary_predictive_rate_parity_diff: 0.0158 - predictive_rate_parity: 0.8993 - binary_accuracy_parity_diff: 0.7836 - accuracy_parity: 0.8918 - false_negative_rate_parity_diff: 0.0095 - false_positive_rate_parity_diff: -0.0095 - binary_negative_predictive_rate_parity_diff: -0.0158 - negative_predictive_rate_parity: 0.8993 - binary

Epoch 17/20
611/611 [==============================] - 645s 1s/step - loss: 0.2175 - accuracy: 0.9052 - true_positives_122: 37708.0000 - true_negatives_80: 33040.0000 - false_positives_98: 3696.0000 - false_negatives_104: 3714.0000 - true_positive_rate: 0.9103 - true_negative_rate: 0.8994 - false_positive_rate: 0.1006 - false_negative_rate: 0.0897 - positive_predicted_value: 0.9107 - false_discovery_rate: 0.0893 - negative_predicted_value: 0.8989 - false_omission_rate: 0.1011 - binary_demographic_parity_diff: 4650.0000 - demographic_parity: 41404.0000 - binary_equalized_odds_diff: -0.0109 - binary_proportional_parity_diff: 0.0595 - proportional_parity: 0.5297 - binary_predictive_rate_parity_diff: 0.0118 - predictive_rate_parity: 0.9107 - binary_accuracy_parity_diff: 0.8104 - accuracy_parity: 0.9052 - false_negative_rate_parity_diff: 0.0109 - false_positive_rate_parity_diff: -0.0109 - binary_negative_predictive_rate_parity_diff: -0.0118 - negative_predictive_rate_parity: 0.9107 - binary

Make folder...


In [10]:
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.892
    
    True positives: 4405.000
    False positives: 286.000
    
    True negatives: 4306.000
    False negatives: 774.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.851
    True negative rate tn/(tn+fp): 0.938
    
    False negative rate fn/(tp+fn): 0.149
    False positive rate fp/(tn+fp): 0.062
    
    Positive predicted value tp/(tp+fp): 0.939
    False discovery rate fp/(tp+fp): 0.061
    
    Negative predicted value tn/(tn+fn): 0.848
    False omission rate fn/(tn+fn): 0.152
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -389.000
    Demographic parity tp+fp: 4691.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.087
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)):

# Transfer Learning

## Transfer learning without augmentation and reweighting

In [4]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221213_134120_Xception_FairFace"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

print(len(model.layers))
model.summary()

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

In [5]:
# Total number of layers: 136 (len(model.layers = 136))
utils.freezeCertainLayers(model, 122)    
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer.h5")

/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 15 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 15 all-reduces with algorithm = nccl, num_packs = 1


2022-12-14 11:24:28.226214: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-14 11:24:29.250709: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-14 11:24:34.388448: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 108s 1s/step - loss: 0.6402 - accuracy: 0.6772 - true_positives_40: 1441.0000 - true_negatives_26: 1211.0000 - false_positives_32: 841.0000 - false_negatives_34: 423.0000 - true_positive_rate: 0.7731 - true_negative_rate: 0.5902 - false_positive_rate: 0.4098 - false_negative_rate: 0.2269 - positive_predicted_value: 0.6315 - false_discovery_rate: 0.3685 - negative_predicted_value: 0.7411 - false_omission_rate: 0.2589 - binary_demographic_parity_diff: 648.0000 - demographic_parity: 2282.0000 - binary_equalized_odds_diff: -0.1829 - binary_proportional_parity_diff: 0.1655 - proportional_parity: 0.5827 - binary_predictive_rate_parity_diff: -0.1097 - predictive_rate_parity: 0.6315 - binary_accuracy_parity_diff: 0.3544 - accuracy_parity: 0.6772 - false_negative_rate_parity_diff: 0.1829 - false_positive_rate_parity_diff: -0.1829 - binary_negative_predictive_rate_parity_diff: 0.1097 - negative_predictive_rate_parity: 0.6315 - binary_specificity_parity_di

Epoch 5/20
31/31 [==============================] - 13s 416ms/step - loss: 0.1503 - accuracy: 0.9760 - true_positives_40: 1835.0000 - true_negatives_26: 1987.0000 - false_positives_32: 65.0000 - false_negatives_34: 29.0000 - true_positive_rate: 0.9844 - true_negative_rate: 0.9683 - false_positive_rate: 0.0317 - false_negative_rate: 0.0156 - positive_predicted_value: 0.9658 - false_discovery_rate: 0.0342 - negative_predicted_value: 0.9856 - false_omission_rate: 0.0144 - binary_demographic_parity_diff: -116.0000 - demographic_parity: 1900.0000 - binary_equalized_odds_diff: -0.0161 - binary_proportional_parity_diff: -0.0296 - proportional_parity: 0.4852 - binary_predictive_rate_parity_diff: -0.0198 - predictive_rate_parity: 0.9658 - binary_accuracy_parity_diff: 0.9520 - accuracy_parity: 0.9760 - false_negative_rate_parity_diff: 0.0161 - false_positive_rate_parity_diff: -0.0161 - binary_negative_predictive_rate_parity_diff: 0.0198 - negative_predictive_rate_parity: 0.9658 - binary_specific

In [6]:
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.778
    
    True positives: 184.000
    False positives: 59.000
    
    True negatives: 199.000
    False negatives: 50.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.786
    True negative rate tn/(tn+fp): 0.771
    
    False negative rate fn/(tp+fn): 0.214
    False positive rate fp/(tn+fp): 0.229
    
    Positive predicted value tp/(tp+fp): 0.757
    False discovery rate fp/(tp+fp): 0.243
    
    Negative predicted value tn/(tn+fn): 0.799
    False omission rate fn/(tn+fn): 0.201
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -6.000
    Demographic parity tp+fp: 243.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.015
    
    Binary proportional

In [7]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221213_134120_Xception_FairFace"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), None, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 145s 2s/step - loss: 0.7042 - accuracy: 0.6546 - true_positives_101: 1301.0000 - true_negatives_66: 1263.0000 - false_positives_81: 780.0000 - false_negatives_86: 573.0000 - true_positive_rate: 0.6942 - true_negative_rate: 0.6182 - false_positive_rate: 0.3818 - false_negative_rate: 0.3058 - positive_predicted_value: 0.6252 - false_discovery_rate: 0.3748 - negative_predicted_value: 0.6879 - false_omission_rate: 0.3121 - binary_demographic_parity_diff: 245.0000 - demographic_parity: 2081.0000 - binary_equalized_odds_diff: -0.0760 - binary_proportional_parity_diff: 0.0625 - proportional_parity: 0.5313 - binary_predictive_rate_parity_diff: -0.0627 - predictive_rate_parity: 0.6252 - binary_accuracy_parity_diff: 0.3092 - accuracy_parity: 0.6546 - false_negative_rate_par

Epoch 5/20
31/31 [==============================] - 37s 1s/step - loss: 0.2804 - accuracy: 0.8836 - true_positives_101: 1643.0000 - true_negatives_66: 1818.0000 - false_positives_81: 225.0000 - false_negatives_86: 231.0000 - true_positive_rate: 0.8767 - true_negative_rate: 0.8899 - false_positive_rate: 0.1101 - false_negative_rate: 0.1233 - positive_predicted_value: 0.8796 - false_discovery_rate: 0.1204 - negative_predicted_value: 0.8873 - false_omission_rate: 0.1127 - binary_demographic_parity_diff: -181.0000 - demographic_parity: 1868.0000 - binary_equalized_odds_diff: 0.0131 - binary_proportional_parity_diff: -0.0462 - proportional_parity: 0.4769 - binary_predictive_rate_parity_diff: -0.0077 - predictive_rate_parity: 0.8796 - binary_accuracy_parity_diff: 0.7672 - accuracy_parity: 0.8836 - false_negative_rate_parity_diff: -0.0131 - false_positive_rate_parity_diff: 0.0131 - binary_negative_predictive_rate_parity_diff: 0.0077 - negative_predictive_rate_parity: 0.8796 - binary_specifici

Epoch 3/20
31/31 [==============================] - 35s 1s/step - loss: 0.3195 - accuracy: 0.8657 - true_positives_101: 1602.0000 - true_negatives_66: 1789.0000 - false_positives_81: 275.0000 - false_negatives_86: 251.0000 - true_positive_rate: 0.8645 - true_negative_rate: 0.8668 - false_positive_rate: 0.1332 - false_negative_rate: 0.1355 - positive_predicted_value: 0.8535 - false_discovery_rate: 0.1465 - negative_predicted_value: 0.8770 - false_omission_rate: 0.1230 - binary_demographic_parity_diff: -163.0000 - demographic_parity: 1877.0000 - binary_equalized_odds_diff: 0.0022 - binary_proportional_parity_diff: -0.0416 - proportional_parity: 0.4792 - binary_predictive_rate_parity_diff: -0.0235 - predictive_rate_parity: 0.8535 - binary_accuracy_parity_diff: 0.7314 - accuracy_parity: 0.8657 - false_negative_rate_parity_diff: -0.0022 - false_positive_rate_parity_diff: 0.0022 - binary_negative_predictive_rate_parity_diff: 0.0235 - negative_predictive_rate_parity: 0.8535 - binary_specifici

Epoch 2/20
31/31 [==============================] - 36s 1s/step - loss: 0.5214 - accuracy: 0.7517 - true_positives_101: 1353.0000 - true_negatives_66: 1592.0000 - false_positives_81: 457.0000 - false_negatives_86: 516.0000 - true_positive_rate: 0.7239 - true_negative_rate: 0.7770 - false_positive_rate: 0.2230 - false_negative_rate: 0.2761 - positive_predicted_value: 0.7475 - false_discovery_rate: 0.2525 - negative_predicted_value: 0.7552 - false_omission_rate: 0.2448 - binary_demographic_parity_diff: -298.0000 - demographic_parity: 1810.0000 - binary_equalized_odds_diff: 0.0530 - binary_proportional_parity_diff: -0.0761 - proportional_parity: 0.4620 - binary_predictive_rate_parity_diff: -0.0077 - predictive_rate_parity: 0.7475 - binary_accuracy_parity_diff: 0.5033 - accuracy_parity: 0.7517 - false_negative_rate_parity_diff: -0.0530 - false_positive_rate_parity_diff: 0.0530 - binary_negative_predictive_rate_parity_diff: 0.0077 - negative_predictive_rate_parity: 0.7475 - binary_specifici

Fold 3...
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 137s 2s/step - loss: 0.7272 - accuracy: 0.6335 - true_positives_101: 1595.0000 - true_negatives_66: 1551.0000 - false_positives_81: 1036.0000 - false_negatives_86: 715.0000 - true_positive_rate: 0.6905 - true_negative_rate: 0.5995 - false_positive_rate: 0.4005 - false_negative_rate: 0.3095 - positive_predicted_value: 0.6062 - false_discovery_rate: 0.3938 - negative_predicted_value: 0.6845 - false_omission_rate: 0.3155 - binary_demographic_parity_diff: 365.0000 - demographic_parity: 2631.0000 - binary_equalized_odds_diff: -0.0909 - binary_proportional_parity_diff: 0.0745 - proportional_parity: 0.5373 - binary_predictive_rate_parity_diff: -0.0782 - predictive_rate_parity: 0.6062 - binary_accuracy_parity_diff: 0.2849 - accuracy_parity: 0.6424 - false_negati

Epoch 5/20
31/31 [==============================] - 37s 1s/step - loss: 0.2362 - accuracy: 0.9002 - true_positives_101: 1658.0000 - true_negatives_66: 1869.0000 - false_positives_81: 201.0000 - false_negatives_86: 190.0000 - true_positive_rate: 0.8972 - true_negative_rate: 0.9029 - false_positive_rate: 0.0971 - false_negative_rate: 0.1028 - positive_predicted_value: 0.8919 - false_discovery_rate: 0.1081 - negative_predicted_value: 0.9077 - false_omission_rate: 0.0923 - binary_demographic_parity_diff: -200.0000 - demographic_parity: 1859.0000 - binary_equalized_odds_diff: 0.0057 - binary_proportional_parity_diff: -0.0510 - proportional_parity: 0.4745 - binary_predictive_rate_parity_diff: -0.0158 - predictive_rate_parity: 0.8919 - binary_accuracy_parity_diff: 0.8004 - accuracy_parity: 0.9002 - false_negative_rate_parity_diff: -0.0057 - false_positive_rate_parity_diff: 0.0057 - binary_negative_predictive_rate_parity_diff: 0.0158 - negative_predictive_rate_parity: 0.8919 - binary_specifici

Epoch 3/20
31/31 [==============================] - 34s 1s/step - loss: 0.3311 - accuracy: 0.8578 - true_positives_101: 1596.0000 - true_negatives_66: 1765.0000 - false_positives_81: 273.0000 - false_negatives_86: 284.0000 - true_positive_rate: 0.8489 - true_negative_rate: 0.8660 - false_positive_rate: 0.1340 - false_negative_rate: 0.1511 - positive_predicted_value: 0.8539 - false_discovery_rate: 0.1461 - negative_predicted_value: 0.8614 - false_omission_rate: 0.1386 - binary_demographic_parity_diff: -180.0000 - demographic_parity: 1869.0000 - binary_equalized_odds_diff: 0.0171 - binary_proportional_parity_diff: -0.0459 - proportional_parity: 0.4770 - binary_predictive_rate_parity_diff: -0.0075 - predictive_rate_parity: 0.8539 - binary_accuracy_parity_diff: 0.7157 - accuracy_parity: 0.8578 - false_negative_rate_parity_diff: -0.0171 - false_positive_rate_parity_diff: 0.0171 - binary_negative_predictive_rate_parity_diff: 0.0075 - negative_predictive_rate_parity: 0.8539 - binary_specifici

Epoch 7/20
31/31 [==============================] - 36s 1s/step - loss: 0.1408 - accuracy: 0.9451 - true_positives_101: 1769.0000 - true_negatives_66: 1934.0000 - false_positives_81: 104.0000 - false_negatives_86: 111.0000 - true_positive_rate: 0.9410 - true_negative_rate: 0.9490 - false_positive_rate: 0.0510 - false_negative_rate: 0.0590 - positive_predicted_value: 0.9445 - false_discovery_rate: 0.0555 - negative_predicted_value: 0.9457 - false_omission_rate: 0.0543 - binary_demographic_parity_diff: -172.0000 - demographic_parity: 1873.0000 - binary_equalized_odds_diff: 0.0080 - binary_proportional_parity_diff: -0.0439 - proportional_parity: 0.4781 - binary_predictive_rate_parity_diff: -0.0012 - predictive_rate_parity: 0.9445 - binary_accuracy_parity_diff: 0.8903 - accuracy_parity: 0.9451 - false_negative_rate_parity_diff: -0.0080 - false_positive_rate_parity_diff: 0.0080 - binary_negative_predictive_rate_parity_diff: 0.0012 - negative_predictive_rate_parity: 0.9445 - binary_specifici

In [8]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.916+/-0.019
    Loss: 0.206+/-0.048
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.913+/-0.021
    True negative rate tn/(tn+fp): 0.920+/-0.019
    
    False negative rate fn/(tp+fn): 0.087+/-0.021
    False positive rate fp/(tn+fp): 0.080+/-0.019
    
    Positive predicted value tp/(tp+fp): 0.912+/-0.021
    False discovery rate fp/(tp+fp): 0.088+/-0.021
    
    Negative predicted value tn/(tn+fn): 0.921+/-0.018
    False omission rate fn/(tn+fn): 0.079+/-0.018
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.047+/-0.010
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.007+/-0.009
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.007+/-0.009
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.656+/-0.037
    Loss: 1.399+/-0.961
    
    Tes

## Transfer learning with augmentation and without reweighting

In [15]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221214_154717_Xception_FairFace_aug"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 136 + 4 augmentation layers 
utils.freezeCertainLayers(model, 128)  
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer_aug.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
31/31 [==============================] - 84s 1s/step - loss: 0.6506 - accuracy: 0.7196 - true_positives_305: 1453.0000 - true_negatives_200: 1365.0000 - false_positives_245: 706.0000 - false_negatives_260: 392.0000 - true_positive_rate: 0.7875 - true_negative_rate: 0.6591 - false_positive_rate: 0.3409 - false_negative_rate: 0.2125 - positive_predicted_value: 0.6730 - false_discovery_rate: 0.3270 - negative_predicted_value: 0.7769 - false_omission_rate: 0.2231 - binary_demographic_parity_diff: 402.0000 - demographic_parity: 2159.0000 - binary_equalized_odds_diff: -0.1284 - binary_proportional_parity_diff: 0.1027 - proportional_parity: 0.5513 - binary_predictive_rate_parity_diff: -0.1039 - predictive_rate_parity: 0.6730 - binary_accuracy_parity_diff: 0.4392 - accuracy_parity: 0.7196 - false_negative_rate_parity_diff: 0.1284 - false_positive_rate_parity_diff: -0.1284 - binary_negative_predictive_rate_parity_diff: 0.1039 - negative_predictive_rate_parity: 0.6730 - binary_specifi

Epoch 5/20
31/31 [==============================] - 10s 337ms/step - loss: 0.4494 - accuracy: 0.7822 - true_positives_305: 1526.0000 - true_negatives_200: 1537.0000 - false_positives_245: 463.0000 - false_negatives_260: 390.0000 - true_positive_rate: 0.7965 - true_negative_rate: 0.7685 - false_positive_rate: 0.2315 - false_negative_rate: 0.2035 - positive_predicted_value: 0.7672 - false_discovery_rate: 0.2328 - negative_predicted_value: 0.7976 - false_omission_rate: 0.2024 - binary_demographic_parity_diff: 62.0000 - demographic_parity: 1989.0000 - binary_equalized_odds_diff: -0.0280 - binary_proportional_parity_diff: 0.0158 - proportional_parity: 0.5079 - binary_predictive_rate_parity_diff: -0.0304 - predictive_rate_parity: 0.7672 - binary_accuracy_parity_diff: 0.5644 - accuracy_parity: 0.7822 - false_negative_rate_parity_diff: 0.0280 - false_positive_rate_parity_diff: -0.0280 - binary_negative_predictive_rate_parity_diff: 0.0304 - negative_predictive_rate_parity: 0.7672 - binary_speci

Epoch 9/20
31/31 [==============================] - 11s 358ms/step - loss: 0.4280 - accuracy: 0.7978 - true_positives_305: 1469.0000 - true_negatives_200: 1655.0000 - false_positives_245: 420.0000 - false_negatives_260: 372.0000 - true_positive_rate: 0.7979 - true_negative_rate: 0.7976 - false_positive_rate: 0.2024 - false_negative_rate: 0.2021 - positive_predicted_value: 0.7777 - false_discovery_rate: 0.2223 - negative_predicted_value: 0.8165 - false_omission_rate: 0.1835 - binary_demographic_parity_diff: -138.0000 - demographic_parity: 1889.0000 - binary_equalized_odds_diff: -3.4553e-04 - binary_proportional_parity_diff: -0.0352 - proportional_parity: 0.4824 - binary_predictive_rate_parity_diff: -0.0388 - predictive_rate_parity: 0.7777 - binary_accuracy_parity_diff: 0.5955 - accuracy_parity: 0.7978 - false_negative_rate_parity_diff: 3.4554e-04 - false_positive_rate_parity_diff: -3.4554e-04 - binary_negative_predictive_rate_parity_diff: 0.0388 - negative_predictive_rate_parity: 0.7777

In [16]:
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.803
    
    True positives: 183.000
    False positives: 46.000
    
    True negatives: 212.000
    False negatives: 51.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.782
    True negative rate tn/(tn+fp): 0.822
    
    False negative rate fn/(tp+fn): 0.218
    False positive rate fp/(tn+fp): 0.178
    
    Positive predicted value tp/(tp+fp): 0.799
    False discovery rate fp/(tp+fp): 0.201
    
    Negative predicted value tn/(tn+fn): 0.806
    False omission rate fn/(tn+fn): 0.194
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -34.000
    Demographic parity tp+fp: 229.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.040
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.06

In [17]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221214_154717_Xception_FairFace_aug"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), None, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
31/31 [==============================] - 134s 2s/step - loss: 0.5884 - accuracy: 0.7128 - true_positives_366: 1439.0000 - true_negatives_240: 1353.0000 - false_positives_294: 713.0000 - false_negatives_312: 412.0000 - true_positive_rate: 0.7774 - true_negative_rate: 0.6549 - false_positive_rate: 0.3451 - false_negative_rate: 0.2226 - positive_predicted_value: 0.6687 - false_discovery_rate: 0.3313 - negative_predicted_value: 0.7666 - false_omission_rate: 0.2334 - binary_demographic_parity_diff: 387.0000 - demographic_parity: 2152.0000 - binary_equalized_odds_diff: -0.1225 - binary_proportional_parity_diff: 0.0988 - proportional_parity: 0.5494 - binary_predictive_rate_parity_diff: -0.0979 - predictive_rate_parity: 0.6687 - binary_accuracy_parity_diff: 0.4256 - accuracy_parity: 0.7128 - false_negative_rate_parity_diff: 0.1225 - false_positive_rate_parity_diff: -0.1225 - binary_negative_predictive_rate_parity_diff: 0.0979 - negative_predictive_rate_parity: 0.6687 - binary_specif

Epoch 5/20
31/31 [==============================] - 35s 1s/step - loss: 0.4059 - accuracy: 0.8172 - true_positives_366: 1498.0000 - true_negatives_240: 1703.0000 - false_positives_294: 363.0000 - false_negatives_312: 353.0000 - true_positive_rate: 0.8093 - true_negative_rate: 0.8243 - false_positive_rate: 0.1757 - false_negative_rate: 0.1907 - positive_predicted_value: 0.8049 - false_discovery_rate: 0.1951 - negative_predicted_value: 0.8283 - false_omission_rate: 0.1717 - binary_demographic_parity_diff: -195.0000 - demographic_parity: 1861.0000 - binary_equalized_odds_diff: 0.0150 - binary_proportional_parity_diff: -0.0498 - proportional_parity: 0.4751 - binary_predictive_rate_parity_diff: -0.0234 - predictive_rate_parity: 0.8049 - binary_accuracy_parity_diff: 0.6344 - accuracy_parity: 0.8172 - false_negative_rate_parity_diff: -0.0150 - false_positive_rate_parity_diff: 0.0150 - binary_negative_predictive_rate_parity_diff: 0.0234 - negative_predictive_rate_parity: 0.8049 - binary_specif

Fold 1...
Epoch 1/20
31/31 [==============================] - 132s 2s/step - loss: 0.6031 - accuracy: 0.7046 - true_positives_366: 1767.0000 - true_negatives_240: 1745.0000 - false_positives_294: 822.0000 - false_negatives_312: 563.0000 - true_positive_rate: 0.7584 - true_negative_rate: 0.6798 - false_positive_rate: 0.3202 - false_negative_rate: 0.2416 - positive_predicted_value: 0.6825 - false_discovery_rate: 0.3175 - negative_predicted_value: 0.7561 - false_omission_rate: 0.2439 - binary_demographic_parity_diff: 281.0000 - demographic_parity: 2589.0000 - binary_equalized_odds_diff: -0.0786 - binary_proportional_parity_diff: 0.0574 - proportional_parity: 0.5287 - binary_predictive_rate_parity_diff: -0.0736 - predictive_rate_parity: 0.6825 - binary_accuracy_parity_diff: 0.4343 - accuracy_parity: 0.7172 - false_negative_rate_parity_diff: 0.0786 - false_positive_rate_parity_diff: -0.0786 - binary_negative_predictive_rate_parity_diff: 0.0736 - negative_predictive_rate_parity: 0.6825 - bin

...

Epoch 7/20
31/31 [==============================] - 35s 1s/step - loss: 0.3955 - accuracy: 0.8201 - true_positives_366: 1529.0000 - true_negatives_240: 1684.0000 - false_positives_294: 334.0000 - false_negatives_312: 371.0000 - true_positive_rate: 0.8047 - true_negative_rate: 0.8345 - false_positive_rate: 0.1655 - false_negative_rate: 0.1953 - positive_predicted_value: 0.8207 - false_discovery_rate: 0.1793 - negative_predicted_value: 0.8195 - false_omission_rate: 0.1805 - binary_demographic_parity_diff: -192.0000 - demographic_parity: 1863.0000 - binary_equalized_odds_diff: 0.0298 - binary_proportional_parity_diff: -0.0490 - proportional_parity: 0.4755 - binary_predictive_rate_parity_diff: 0.0013 - predictive_rate_parity: 0.8207 - binary_accuracy_parity_diff: 0.6401 - accuracy_parity: 0.8201 - false_negative_rate_parity_diff: -0.0298 - false_positive_rate_parity_diff: 0.0298 - binary_negative_predictive_rate_parity_diff: -0.0013 - negative_predictive_rate_parity: 0.8207 - binary_specif

Epoch 11/20
31/31 [==============================] - 36s 1s/step - loss: 0.3067 - accuracy: 0.8726 - true_positives_366: 1635.0000 - true_negatives_240: 1784.0000 - false_positives_294: 234.0000 - false_negatives_312: 265.0000 - true_positive_rate: 0.8605 - true_negative_rate: 0.8840 - false_positive_rate: 0.1160 - false_negative_rate: 0.1395 - positive_predicted_value: 0.8748 - false_discovery_rate: 0.1252 - negative_predicted_value: 0.8707 - false_omission_rate: 0.1293 - binary_demographic_parity_diff: -180.0000 - demographic_parity: 1869.0000 - binary_equalized_odds_diff: 0.0235 - binary_proportional_parity_diff: -0.0459 - proportional_parity: 0.4770 - binary_predictive_rate_parity_diff: 0.0041 - predictive_rate_parity: 0.8748 - binary_accuracy_parity_diff: 0.7453 - accuracy_parity: 0.8726 - false_negative_rate_parity_diff: -0.0235 - false_positive_rate_parity_diff: 0.0235 - binary_negative_predictive_rate_parity_diff: -0.0041 - negative_predictive_rate_parity: 0.8748 - binary_speci

Epoch 3/20
31/31 [==============================] - 35s 1s/step - loss: 0.4509 - accuracy: 0.7915 - true_positives_366: 1435.0000 - true_negatives_240: 1666.0000 - false_positives_294: 394.0000 - false_negatives_312: 423.0000 - true_positive_rate: 0.7723 - true_negative_rate: 0.8087 - false_positive_rate: 0.1913 - false_negative_rate: 0.2277 - positive_predicted_value: 0.7846 - false_discovery_rate: 0.2154 - negative_predicted_value: 0.7975 - false_omission_rate: 0.2025 - binary_demographic_parity_diff: -260.0000 - demographic_parity: 1829.0000 - binary_equalized_odds_diff: 0.0364 - binary_proportional_parity_diff: -0.0664 - proportional_parity: 0.4668 - binary_predictive_rate_parity_diff: -0.0129 - predictive_rate_parity: 0.7846 - binary_accuracy_parity_diff: 0.5830 - accuracy_parity: 0.7915 - false_negative_rate_parity_diff: -0.0364 - false_positive_rate_parity_diff: 0.0364 - binary_negative_predictive_rate_parity_diff: 0.0129 - negative_predictive_rate_parity: 0.7846 - binary_specif

Epoch 7/20
31/31 [==============================] - 34s 1s/step - loss: 0.3496 - accuracy: 0.8423 - true_positives_366: 1539.0000 - true_negatives_240: 1761.0000 - false_positives_294: 299.0000 - false_negatives_312: 319.0000 - true_positive_rate: 0.8283 - true_negative_rate: 0.8549 - false_positive_rate: 0.1451 - false_negative_rate: 0.1717 - positive_predicted_value: 0.8373 - false_discovery_rate: 0.1627 - negative_predicted_value: 0.8466 - false_omission_rate: 0.1534 - binary_demographic_parity_diff: -242.0000 - demographic_parity: 1838.0000 - binary_equalized_odds_diff: 0.0265 - binary_proportional_parity_diff: -0.0618 - proportional_parity: 0.4691 - binary_predictive_rate_parity_diff: -0.0093 - predictive_rate_parity: 0.8373 - binary_accuracy_parity_diff: 0.6845 - accuracy_parity: 0.8423 - false_negative_rate_parity_diff: -0.0265 - false_positive_rate_parity_diff: 0.0265 - binary_negative_predictive_rate_parity_diff: 0.0093 - negative_predictive_rate_parity: 0.8373 - binary_specif

Epoch 11/20
31/31 [==============================] - 34s 1s/step - loss: 0.2991 - accuracy: 0.8721 - true_positives_366: 1602.0000 - true_negatives_240: 1815.0000 - false_positives_294: 245.0000 - false_negatives_312: 256.0000 - true_positive_rate: 0.8622 - true_negative_rate: 0.8811 - false_positive_rate: 0.1189 - false_negative_rate: 0.1378 - positive_predicted_value: 0.8674 - false_discovery_rate: 0.1326 - negative_predicted_value: 0.8764 - false_omission_rate: 0.1236 - binary_demographic_parity_diff: -224.0000 - demographic_parity: 1847.0000 - binary_equalized_odds_diff: 0.0189 - binary_proportional_parity_diff: -0.0572 - proportional_parity: 0.4714 - binary_predictive_rate_parity_diff: -0.0090 - predictive_rate_parity: 0.8674 - binary_accuracy_parity_diff: 0.7443 - accuracy_parity: 0.8721 - false_negative_rate_parity_diff: -0.0189 - false_positive_rate_parity_diff: 0.0189 - binary_negative_predictive_rate_parity_diff: 0.0090 - negative_predictive_rate_parity: 0.8674 - binary_speci

In [18]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception_aug"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.865+/-0.019
    Loss: 0.311+/-0.041
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.854+/-0.022
    True negative rate tn/(tn+fp): 0.875+/-0.017
    
    False negative rate fn/(tp+fn): 0.146+/-0.022
    False positive rate fp/(tn+fp): 0.125+/-0.017
    
    Positive predicted value tp/(tp+fp): 0.862+/-0.019
    False discovery rate fp/(tp+fp): 0.138+/-0.019
    
    Negative predicted value tn/(tn+fn): 0.869+/-0.019
    False omission rate fn/(tn+fn): 0.131+/-0.019
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.056+/-0.017
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.021+/-0.013
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.021+/-0.013
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.725+/-0.093
    Loss: 1.513+/-1.683
    
    Tes

## Transfer learning without augmentation and with reweighting

In [4]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221213_162449_Xception_FairFace_re"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 136    
utils.freezeCertainLayers(model, 122)   
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer_re"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer_re.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 15 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 15 all-reduces with algorithm = nccl, num_packs = 1


2022-12-14 08:41:19.742575: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-14 08:41:21.620845: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-14 08:41:25.197918: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 96s 1s/step - loss: 0.6303 - accuracy: 0.6902 - true_positives_40: 1406.0000 - true_negatives_26: 1297.0000 - false_positives_32: 755.0000 - false_negatives_34: 458.0000 - true_positive_rate: 0.7543 - true_negative_rate: 0.6321 - false_positive_rate: 0.3679 - false_negative_rate: 0.2457 - positive_predicted_value: 0.6506 - false_discovery_rate: 0.3494 - negative_predicted_value: 0.7390 - false_omission_rate: 0.2610 - binary_demographic_parity_diff: 406.0000 - demographic_parity: 2161.0000 - binary_equalized_odds_diff: -0.1222 - binary_proportional_parity_diff: 0.1037 - proportional_parity: 0.5518 - binary_predictive_rate_parity_diff: -0.0884 - predictive_rate_parity: 0.6506 - binary_accuracy_parity_diff: 0.3805 - accuracy_parity: 0.6902 - false_negative_rate_parity_diff: 0.1222 - false_positive_rate_parity_diff: -0.1222 - binary_negative_predictive_rate_parity_diff: 0.0884 - negative_predictive_rate_parity: 0.6506 - binary_specificity_parity_dif

Make folder...
Name directory: 20221213_162449_Xception_FairFace_re


In [5]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.785
    
    True positives: 156.000
    False positives: 28.000
    
    True negatives: 230.000
    False negatives: 78.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.667
    True negative rate tn/(tn+fp): 0.891
    
    False negative rate fn/(tp+fn): 0.333
    False positive rate fp/(tn+fp): 0.109
    
    Positive predicted value tp/(tp+fp): 0.848
    False discovery rate fp/(tp+fp): 0.152
    
    Negative predicted value tn/(tn+fn): 0.747
    False omission rate fn/(tn+fn): 0.253
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -124.000
    Demographic parity tp+fp: 184.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.225
    
    Binary proportio

In [6]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221213_162449_Xception_FairFace_re"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), class_weight, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 126s 2s/step - loss: 0.7041 - accuracy: 0.6525 - true_positives_101: 1309.0000 - true_negatives_66: 1247.0000 - false_positives_81: 799.0000 - false_negatives_86: 562.0000 - true_positive_rate: 0.6996 - true_negative_rate: 0.6095 - false_positive_rate: 0.3905 - false_negative_rate: 0.3004 - positive_predicted_value: 0.6210 - false_discovery_rate: 0.3790 - negative_predicted_value: 0.6893 - false_omission_rate: 0.3107 - binary_demographic_parity_diff: 299.0000 - demographic_parity: 2108.0000 - binary_equalized_odds_diff: -0.0901 - binary_proportional_parity_diff: 0.0763 - proportional_parity: 0.5382 - binary_predictive_rate_parity_diff: -0.0684 - predictive_rate_parity: 0.6210 - binary_accuracy_parity_diff: 0.3051 - accuracy_parity: 0.6525 - false_negative_rate_par

Epoch 5/20
31/31 [==============================] - 37s 1s/step - loss: 0.2122 - accuracy: 0.9071 - true_positives_101: 1689.0000 - true_negatives_66: 1864.0000 - false_positives_81: 182.0000 - false_negatives_86: 182.0000 - true_positive_rate: 0.9027 - true_negative_rate: 0.9110 - false_positive_rate: 0.0890 - false_negative_rate: 0.0973 - positive_predicted_value: 0.9027 - false_discovery_rate: 0.0973 - negative_predicted_value: 0.9110 - false_omission_rate: 0.0890 - binary_demographic_parity_diff: -175.0000 - demographic_parity: 1871.0000 - binary_equalized_odds_diff: 0.0083 - binary_proportional_parity_diff: -0.0447 - proportional_parity: 0.4777 - binary_predictive_rate_parity_diff: -0.0083 - predictive_rate_parity: 0.9027 - binary_accuracy_parity_diff: 0.8141 - accuracy_parity: 0.9071 - false_negative_rate_parity_diff: -0.0083 - false_positive_rate_parity_diff: 0.0083 - binary_negative_predictive_rate_parity_diff: 0.0083 - negative_predictive_rate_parity: 0.9027 - binary_specifici

Epoch 3/20
31/31 [==============================] - 40s 1s/step - loss: 0.2452 - accuracy: 0.8956 - true_positives_101: 1652.0000 - true_negatives_66: 1856.0000 - false_positives_81: 221.0000 - false_negatives_86: 188.0000 - true_positive_rate: 0.8978 - true_negative_rate: 0.8936 - false_positive_rate: 0.1064 - false_negative_rate: 0.1022 - positive_predicted_value: 0.8820 - false_discovery_rate: 0.1180 - negative_predicted_value: 0.9080 - false_omission_rate: 0.0920 - binary_demographic_parity_diff: -171.0000 - demographic_parity: 1873.0000 - binary_equalized_odds_diff: -0.0042 - binary_proportional_parity_diff: -0.0437 - proportional_parity: 0.4782 - binary_predictive_rate_parity_diff: -0.0260 - predictive_rate_parity: 0.8820 - binary_accuracy_parity_diff: 0.7912 - accuracy_parity: 0.8956 - false_negative_rate_parity_diff: 0.0042 - false_positive_rate_parity_diff: -0.0042 - binary_negative_predictive_rate_parity_diff: 0.0260 - negative_predictive_rate_parity: 0.8820 - binary_specific

...

Epoch 11/20
31/31 [==============================] - 37s 1s/step - loss: 0.0670 - accuracy: 0.9765 - true_positives_101: 1794.0000 - true_negatives_66: 2031.0000 - false_positives_81: 46.0000 - false_negatives_86: 46.0000 - true_positive_rate: 0.9750 - true_negative_rate: 0.9779 - false_positive_rate: 0.0221 - false_negative_rate: 0.0250 - positive_predicted_value: 0.9750 - false_discovery_rate: 0.0250 - negative_predicted_value: 0.9779 - false_omission_rate: 0.0221 - binary_demographic_parity_diff: -237.0000 - demographic_parity: 1840.0000 - binary_equalized_odds_diff: 0.0029 - binary_proportional_parity_diff: -0.0605 - proportional_parity: 0.4697 - binary_predictive_rate_parity_diff: -0.0029 - predictive_rate_parity: 0.9750 - binary_accuracy_parity_diff: 0.9530 - accuracy_parity: 0.9765 - false_negative_rate_parity_diff: -0.0029 - false_positive_rate_parity_diff: 0.0029 - binary_negative_predictive_rate_parity_diff: 0.0029 - negative_predictive_rate_parity: 0.9750 - binary_specificit

Epoch 4/20
31/31 [==============================] - 37s 1s/step - loss: 0.2362 - accuracy: 0.9068 - true_positives_101: 1720.0000 - true_negatives_66: 1833.0000 - false_positives_81: 205.0000 - false_negatives_86: 160.0000 - true_positive_rate: 0.9149 - true_negative_rate: 0.8994 - false_positive_rate: 0.1006 - false_negative_rate: 0.0851 - positive_predicted_value: 0.8935 - false_discovery_rate: 0.1065 - negative_predicted_value: 0.9197 - false_omission_rate: 0.0803 - binary_demographic_parity_diff: -68.0000 - demographic_parity: 1925.0000 - binary_equalized_odds_diff: -0.0155 - binary_proportional_parity_diff: -0.0174 - proportional_parity: 0.4913 - binary_predictive_rate_parity_diff: -0.0262 - predictive_rate_parity: 0.8935 - binary_accuracy_parity_diff: 0.8137 - accuracy_parity: 0.9068 - false_negative_rate_parity_diff: 0.0155 - false_positive_rate_parity_diff: -0.0155 - binary_negative_predictive_rate_parity_diff: 0.0262 - negative_predictive_rate_parity: 0.8935 - binary_specifici

Epoch 2/20
31/31 [==============================] - 38s 1s/step - loss: 0.3817 - accuracy: 0.8308 - true_positives_101: 1557.0000 - true_negatives_66: 1698.0000 - false_positives_81: 347.0000 - false_negatives_86: 316.0000 - true_positive_rate: 0.8313 - true_negative_rate: 0.8303 - false_positive_rate: 0.1697 - false_negative_rate: 0.1687 - positive_predicted_value: 0.8178 - false_discovery_rate: 0.1822 - negative_predicted_value: 0.8431 - false_omission_rate: 0.1569 - binary_demographic_parity_diff: -110.0000 - demographic_parity: 1904.0000 - binary_equalized_odds_diff: -9.6887e-04 - binary_proportional_parity_diff: -0.0281 - proportional_parity: 0.4860 - binary_predictive_rate_parity_diff: -0.0253 - predictive_rate_parity: 0.8178 - binary_accuracy_parity_diff: 0.6616 - accuracy_parity: 0.8308 - false_negative_rate_parity_diff: 9.6884e-04 - false_positive_rate_parity_diff: -9.6884e-04 - binary_negative_predictive_rate_parity_diff: 0.0253 - negative_predictive_rate_parity: 0.8178 - bin

Fold 4...
Epoch 1/20
31/31 [==============================] - 126s 2s/step - loss: 0.6799 - accuracy: 0.6779 - true_positives_101: 1486.0000 - true_negatives_66: 1799.0000 - false_positives_81: 780.0000 - false_negatives_86: 832.0000 - true_positive_rate: 0.6411 - true_negative_rate: 0.6976 - false_positive_rate: 0.3024 - false_negative_rate: 0.3589 - positive_predicted_value: 0.6558 - false_discovery_rate: 0.3442 - negative_predicted_value: 0.6838 - false_omission_rate: 0.3162 - binary_demographic_parity_diff: -365.0000 - demographic_parity: 2266.0000 - binary_equalized_odds_diff: 0.0565 - binary_proportional_parity_diff: -0.0745 - proportional_parity: 0.4627 - binary_predictive_rate_parity_diff: -0.0280 - predictive_rate_parity: 0.6558 - binary_accuracy_parity_diff: 0.3416 - accuracy_parity: 0.6708 - false_negative_rate_parity_diff: -0.0565 - false_positive_rate_parity_diff: 0.0565 - binary_negative_predictive_rate_parity_diff: 0.0280 - negative_predictive_rate_parity: 0.6558 - binar

Epoch 5/20
31/31 [==============================] - 37s 1s/step - loss: 0.1830 - accuracy: 0.9321 - true_positives_101: 1725.0000 - true_negatives_66: 1927.0000 - false_positives_81: 131.0000 - false_negatives_86: 135.0000 - true_positive_rate: 0.9274 - true_negative_rate: 0.9363 - false_positive_rate: 0.0637 - false_negative_rate: 0.0726 - positive_predicted_value: 0.9294 - false_discovery_rate: 0.0706 - negative_predicted_value: 0.9345 - false_omission_rate: 0.0655 - binary_demographic_parity_diff: -206.0000 - demographic_parity: 1856.0000 - binary_equalized_odds_diff: 0.0089 - binary_proportional_parity_diff: -0.0526 - proportional_parity: 0.4737 - binary_predictive_rate_parity_diff: -0.0051 - predictive_rate_parity: 0.9294 - binary_accuracy_parity_diff: 0.8642 - accuracy_parity: 0.9321 - false_negative_rate_parity_diff: -0.0089 - false_positive_rate_parity_diff: 0.0089 - binary_negative_predictive_rate_parity_diff: 0.0051 - negative_predictive_rate_parity: 0.9294 - binary_specifici

In [7]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception_re"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.937+/-0.023
    Loss: 0.160+/-0.053
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.940+/-0.023
    True negative rate tn/(tn+fp): 0.935+/-0.025
    
    False negative rate fn/(tp+fn): 0.060+/-0.023
    False positive rate fp/(tn+fp): 0.065+/-0.025
    
    Positive predicted value tp/(tp+fp): 0.930+/-0.026
    False discovery rate fp/(tp+fp): 0.070+/-0.026
    
    Negative predicted value tn/(tn+fn): 0.945+/-0.021
    False omission rate fn/(tn+fn): 0.055+/-0.021
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.037+/-0.012
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.005+/-0.006
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): 0.005+/-0.006
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.665+/-0.060
    Loss: 1.529+/-0.661
    
    Tes

## Transfer learning with augmentation and reweighting

In [5]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221214_154717_Xception_FairFace_aug"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 136 + 4 augmentation layers 
utils.freezeCertainLayers(model, 128)    
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer_re_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer_re_aug.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 12 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 12 all-reduces with algorithm = nccl, num_packs = 1


2022-12-15 10:48:07.556828: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-15 10:48:09.313704: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-15 10:48:16.169885: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


62/62 [==============================] - 101s 688ms/step - loss: 0.6196 - accuracy: 0.7186 - true_positives_60: 1460.0000 - true_negatives_39: 1354.0000 - false_positives_48: 698.0000 - false_negatives_51: 404.0000 - true_positive_rate: 0.7833 - true_negative_rate: 0.6598 - false_positive_rate: 0.3402 - false_negative_rate: 0.2167 - positive_predicted_value: 0.6766 - false_discovery_rate: 0.3234 - negative_predicted_value: 0.7702 - false_omission_rate: 0.2298 - binary_demographic_parity_diff: 400.0000 - demographic_parity: 2158.0000 - binary_equalized_odds_diff: -0.1234 - binary_proportional_parity_diff: 0.1021 - proportional_parity: 0.5511 - binary_predictive_rate_parity_diff: -0.0936 - predictive_rate_parity: 0.6766 - binary_accuracy_parity_diff: 0.4372 - accuracy_parity: 0.7186 - false_negative_rate_parity_diff: 0.1234 - false_positive_rate_parity_diff: -0.1234 - binary_negative_predictive_rate_parity_diff: 0.0936 - negative_predictive_rate_parity: 0.6766 - binary_specificity_parity

Epoch 5/20
62/62 [==============================] - 20s 328ms/step - loss: 0.4702 - accuracy: 0.7748 - true_positives_60: 1485.0000 - true_negatives_39: 1549.0000 - false_positives_48: 503.0000 - false_negatives_51: 379.0000 - true_positive_rate: 0.7967 - true_negative_rate: 0.7549 - false_positive_rate: 0.2451 - false_negative_rate: 0.2033 - positive_predicted_value: 0.7470 - false_discovery_rate: 0.2530 - negative_predicted_value: 0.8034 - false_omission_rate: 0.1966 - binary_demographic_parity_diff: 60.0000 - demographic_parity: 1988.0000 - binary_equalized_odds_diff: -0.0418 - binary_proportional_parity_diff: 0.0153 - proportional_parity: 0.5077 - binary_predictive_rate_parity_diff: -0.0564 - predictive_rate_parity: 0.7470 - binary_accuracy_parity_diff: 0.5495 - accuracy_parity: 0.7748 - false_negative_rate_parity_diff: 0.0418 - false_positive_rate_parity_diff: -0.0418 - binary_negative_predictive_rate_parity_diff: 0.0564 - negative_predictive_rate_parity: 0.7470 - binary_specifici

Make folder...
Name directory: 20221215_105133_Xception_transfer_re_aug


In [6]:
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.809
    
    True positives: 162.000
    False positives: 22.000
    
    True negatives: 236.000
    False negatives: 72.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.692
    True negative rate tn/(tn+fp): 0.915
    
    False negative rate fn/(tp+fn): 0.308
    False positive rate fp/(tn+fp): 0.085
    
    Positive predicted value tp/(tp+fp): 0.880
    False discovery rate fp/(tp+fp): 0.120
    
    Negative predicted value tn/(tn+fn): 0.766
    False omission rate fn/(tn+fn): 0.234
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -124.000
    Demographic parity tp+fp: 184.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.222
    
    Binary proportio

In [7]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221214_154717_Xception_FairFace_aug"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), class_weight, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 134s 2s/step - loss: 0.5729 - accuracy: 0.7148 - true_positives_121: 1490.0000 - true_negatives_79: 1310.0000 - false_positives_97: 731.0000 - false_negatives_103: 386.0000 - true_positive_rate: 0.7942 - true_negative_rate: 0.6418 - false_positive_rate: 0.3582 - false_negative_rate: 0.2058 - positive_predicted_value: 0.6709 - false_discovery_rate: 0.3291 - negative_predicted_value: 0.7724 - false_omission_rate: 0.2276 - binary_demographic_parity_diff: 525.0000 - demographic_parity: 2221.0000 - binary_equalized_odds_diff: -0.1524 - binary_proportional_parity_diff: 0.1340 - proportional_parity: 0.5670 - binary_predictive_rate_parity_diff: -0.1015 - predictive_rate_parity: 0.6709 - binary_accuracy_parity_diff: 0.4297 - accuracy_parity: 0.7148 - false_negative_rate_pa

Epoch 5/20
31/31 [==============================] - 39s 1s/step - loss: 0.4009 - accuracy: 0.8141 - true_positives_121: 1519.0000 - true_negatives_79: 1670.0000 - false_positives_97: 371.0000 - false_negatives_103: 357.0000 - true_positive_rate: 0.8097 - true_negative_rate: 0.8182 - false_positive_rate: 0.1818 - false_negative_rate: 0.1903 - positive_predicted_value: 0.8037 - false_discovery_rate: 0.1963 - negative_predicted_value: 0.8239 - false_omission_rate: 0.1761 - binary_demographic_parity_diff: -137.0000 - demographic_parity: 1890.0000 - binary_equalized_odds_diff: 0.0085 - binary_proportional_parity_diff: -0.0350 - proportional_parity: 0.4825 - binary_predictive_rate_parity_diff: -0.0202 - predictive_rate_parity: 0.8037 - binary_accuracy_parity_diff: 0.6283 - accuracy_parity: 0.8141 - false_negative_rate_parity_diff: -0.0085 - false_positive_rate_parity_diff: 0.0085 - binary_negative_predictive_rate_parity_diff: 0.0202 - negative_predictive_rate_parity: 0.8037 - binary_specific

Epoch 9/20
31/31 [==============================] - 39s 1s/step - loss: 0.3254 - accuracy: 0.8606 - true_positives_121: 1611.0000 - true_negatives_79: 1760.0000 - false_positives_97: 281.0000 - false_negatives_103: 265.0000 - true_positive_rate: 0.8587 - true_negative_rate: 0.8623 - false_positive_rate: 0.1377 - false_negative_rate: 0.1413 - positive_predicted_value: 0.8515 - false_discovery_rate: 0.1485 - negative_predicted_value: 0.8691 - false_omission_rate: 0.1309 - binary_demographic_parity_diff: -133.0000 - demographic_parity: 1892.0000 - binary_equalized_odds_diff: 0.0036 - binary_proportional_parity_diff: -0.0340 - proportional_parity: 0.4830 - binary_predictive_rate_parity_diff: -0.0177 - predictive_rate_parity: 0.8515 - binary_accuracy_parity_diff: 0.7212 - accuracy_parity: 0.8606 - false_negative_rate_parity_diff: -0.0036 - false_positive_rate_parity_diff: 0.0036 - binary_negative_predictive_rate_parity_diff: 0.0177 - negative_predictive_rate_parity: 0.8515 - binary_specific

...

Epoch 7/20
31/31 [==============================] - 40s 1s/step - loss: 0.3630 - accuracy: 0.8420 - true_positives_121: 1563.0000 - true_negatives_79: 1736.0000 - false_positives_97: 315.0000 - false_negatives_103: 304.0000 - true_positive_rate: 0.8372 - true_negative_rate: 0.8464 - false_positive_rate: 0.1536 - false_negative_rate: 0.1628 - positive_predicted_value: 0.8323 - false_discovery_rate: 0.1677 - negative_predicted_value: 0.8510 - false_omission_rate: 0.1490 - binary_demographic_parity_diff: -162.0000 - demographic_parity: 1878.0000 - binary_equalized_odds_diff: 0.0092 - binary_proportional_parity_diff: -0.0413 - proportional_parity: 0.4793 - binary_predictive_rate_parity_diff: -0.0187 - predictive_rate_parity: 0.8323 - binary_accuracy_parity_diff: 0.6840 - accuracy_parity: 0.8420 - false_negative_rate_parity_diff: -0.0092 - false_positive_rate_parity_diff: 0.0092 - binary_negative_predictive_rate_parity_diff: 0.0187 - negative_predictive_rate_parity: 0.8323 - binary_specific

Epoch 4/20
31/31 [==============================] - 39s 1s/step - loss: 0.4552 - accuracy: 0.7989 - true_positives_121: 1479.0000 - true_negatives_79: 1651.0000 - false_positives_97: 398.0000 - false_negatives_103: 390.0000 - true_positive_rate: 0.7913 - true_negative_rate: 0.8058 - false_positive_rate: 0.1942 - false_negative_rate: 0.2087 - positive_predicted_value: 0.7880 - false_discovery_rate: 0.2120 - negative_predicted_value: 0.8089 - false_omission_rate: 0.1911 - binary_demographic_parity_diff: -164.0000 - demographic_parity: 1877.0000 - binary_equalized_odds_diff: 0.0144 - binary_proportional_parity_diff: -0.0419 - proportional_parity: 0.4791 - binary_predictive_rate_parity_diff: -0.0210 - predictive_rate_parity: 0.7880 - binary_accuracy_parity_diff: 0.5978 - accuracy_parity: 0.7989 - false_negative_rate_parity_diff: -0.0144 - false_positive_rate_parity_diff: 0.0144 - binary_negative_predictive_rate_parity_diff: 0.0210 - negative_predictive_rate_parity: 0.7880 - binary_specific

Epoch 8/20
31/31 [==============================] - 39s 1s/step - loss: 0.3730 - accuracy: 0.8361 - true_positives_121: 1564.0000 - true_negatives_79: 1712.0000 - false_positives_97: 337.0000 - false_negatives_103: 305.0000 - true_positive_rate: 0.8368 - true_negative_rate: 0.8355 - false_positive_rate: 0.1645 - false_negative_rate: 0.1632 - positive_predicted_value: 0.8227 - false_discovery_rate: 0.1773 - negative_predicted_value: 0.8488 - false_omission_rate: 0.1512 - binary_demographic_parity_diff: -116.0000 - demographic_parity: 1901.0000 - binary_equalized_odds_diff: -0.0013 - binary_proportional_parity_diff: -0.0296 - proportional_parity: 0.4852 - binary_predictive_rate_parity_diff: -0.0261 - predictive_rate_parity: 0.8227 - binary_accuracy_parity_diff: 0.6723 - accuracy_parity: 0.8361 - false_negative_rate_parity_diff: 0.0013 - false_positive_rate_parity_diff: -0.0013 - binary_negative_predictive_rate_parity_diff: 0.0261 - negative_predictive_rate_parity: 0.8227 - binary_specifi

Epoch 2/20
31/31 [==============================] - 39s 1s/step - loss: 0.4948 - accuracy: 0.7695 - true_positives_121: 1472.0000 - true_negatives_79: 1543.0000 - false_positives_97: 493.0000 - false_negatives_103: 410.0000 - true_positive_rate: 0.7821 - true_negative_rate: 0.7579 - false_positive_rate: 0.2421 - false_negative_rate: 0.2179 - positive_predicted_value: 0.7491 - false_discovery_rate: 0.2509 - negative_predicted_value: 0.7901 - false_omission_rate: 0.2099 - binary_demographic_parity_diff: 12.0000 - demographic_parity: 1965.0000 - binary_equalized_odds_diff: -0.0243 - binary_proportional_parity_diff: 0.0031 - proportional_parity: 0.5015 - binary_predictive_rate_parity_diff: -0.0410 - predictive_rate_parity: 0.7491 - binary_accuracy_parity_diff: 0.5391 - accuracy_parity: 0.7695 - false_negative_rate_parity_diff: 0.0243 - false_positive_rate_parity_diff: -0.0243 - binary_negative_predictive_rate_parity_diff: 0.0410 - negative_predictive_rate_parity: 0.7491 - binary_specificit

Epoch 6/20
31/31 [==============================] - 39s 1s/step - loss: 0.4095 - accuracy: 0.8185 - true_positives_121: 1541.0000 - true_negatives_79: 1666.0000 - false_positives_97: 370.0000 - false_negatives_103: 341.0000 - true_positive_rate: 0.8188 - true_negative_rate: 0.8183 - false_positive_rate: 0.1817 - false_negative_rate: 0.1812 - positive_predicted_value: 0.8064 - false_discovery_rate: 0.1936 - negative_predicted_value: 0.8301 - false_omission_rate: 0.1699 - binary_demographic_parity_diff: -96.0000 - demographic_parity: 1911.0000 - binary_equalized_odds_diff: -5.3865e-04 - binary_proportional_parity_diff: -0.0245 - proportional_parity: 0.4877 - binary_predictive_rate_parity_diff: -0.0237 - predictive_rate_parity: 0.8064 - binary_accuracy_parity_diff: 0.6371 - accuracy_parity: 0.8185 - false_negative_rate_parity_diff: 5.3866e-04 - false_positive_rate_parity_diff: -5.3866e-04 - binary_negative_predictive_rate_parity_diff: 0.0237 - negative_predictive_rate_parity: 0.8064 - bin

In [8]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception_re_aug"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.847+/-0.014
    Loss: 0.351+/-0.027
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.845+/-0.019
    True negative rate tn/(tn+fp): 0.848+/-0.011
    
    False negative rate fn/(tp+fn): 0.155+/-0.019
    False positive rate fp/(tn+fp): 0.152+/-0.011
    
    Positive predicted value tp/(tp+fp): 0.835+/-0.014
    False discovery rate fp/(tp+fp): 0.165+/-0.014
    
    Negative predicted value tn/(tn+fn): 0.858+/-0.015
    False omission rate fn/(tn+fn): 0.142+/-0.015
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.036+/-0.016
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.003+/-0.012
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.003+/-0.012
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.731+/-0.078
    Loss: 3.311+/-5.194
    
    Tes

# Fine-tuning
Fine tuning

In [61]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "20221214_154717_Xception_FairFace_aug"

model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
"""
with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )
"""
# Total number of layers: 136 + 4 augmentation layers 
utils.freezeCertainLayers(model, 128)

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [62]:
print(len(model.layers))
model.summary()

138
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 299, 299, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 299, 299, 3)  0           ['sequential[0][0]']             
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['rescaling[0][0]']      

In [63]:
model2 = tf.keras.Model(model.input, model.layers[-5].output)

In [64]:
print(len(model2.layers))
model2.summary()

134
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 299, 299, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 299, 299, 3)  0           ['sequential[0][0]']             
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['rescaling[0][0]']    

In [65]:
x = model2(model2.input, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

In [66]:
model = tf.keras.Model(model2.input, outputs)
print(len(model.layers))
model.summary()

6
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 model_6 (Functional)        (None, 10, 10, 2048)      20861480  
                                                                 
 avg_pool (GlobalAveragePool  (None, 2048)             0         
 ing2D)                                                          
                                                                 
 batch_normalization_3 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 20

In [68]:
with strategy.scope():
    model = model.copy()   
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer_re_aug_fine"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer_re_aug_fine.h5")

AttributeError: 'Functional' object has no attribute 'copy'

In [6]:
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.809
    
    True positives: 162.000
    False positives: 22.000
    
    True negatives: 236.000
    False negatives: 72.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.692
    True negative rate tn/(tn+fp): 0.915
    
    False negative rate fn/(tp+fn): 0.308
    False positive rate fp/(tn+fp): 0.085
    
    Positive predicted value tp/(tp+fp): 0.880
    False discovery rate fp/(tp+fp): 0.120
    
    Negative predicted value tn/(tn+fn): 0.766
    False omission rate fn/(tn+fn): 0.234
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -124.000
    Demographic parity tp+fp: 184.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.222
    
    Binary proportio